In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.5/661.5 kB 12.5 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... error
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [50 lines of output]
      + meson setup /tmp/pip-install-2tsimoeg/pycairo_dbe513ed5ed14c6fb225d98d225e4969 /tmp/pip-install-2tsimoeg/pycairo_dbe513ed5ed14c6fb225d98d225e4969/.mesonpy-mwy8pxg8 -Dbui

In [2]:
!pip install datasets
!pip install transformers
!pip install einops
!pip install trl
!pip install huggingface_hub
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"
!pip -q uninstall transformers -y
!pip -q install transformers==4.47.1

  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached pyarrow-18.1.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.2.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
  Using cached aiohttp-3.11.11-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached aiohappyeyeballs-2.4.4-py3-none-any.whl.metadata (6.1 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached frozenlist-1.5.0-cp310-cp310-manylinux_2_5_x86

### Loading in model from Unsloth

In [3]:
import torch
from unsloth import FastLanguageModel

# Clearing GPU memory cache
torch.cuda.empty_cache()

# Set model device type to 'cuda'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Using unsloth fast library
model, tokenizer =  FastLanguageModel.from_pretrained("unsloth/mistral-7b-v0.3", 
                                                      dtype=None, 
                                                      load_in_4bit=True )

tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = "alpaca"

# Using LoRA adapters to only update part of the model's parameters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",   
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False, 
    loftq_config = None, 
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
🦥 Unsloth Zoo will now patch everything to make training faster!
Using device: cuda:0
==((====))==  Unsloth 2025.1.5: Fast Mistral patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA H100 PCIe. Max memory: 79.097 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Unsloth 2025.1.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
train_dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
train_dataset = train_dataset.map(formatting_prompts_func, batched = True,)## Dataset for training

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

## SFT Training

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

training_args = TrainingArguments(
        output_dir="Mistral_7B-SFT",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed = 3407,
    )

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    dataset_text_field = "text",
    max_seq_length=2048,
    dataset_num_proc=2,
    packing=False, # Can make training 5x faster for short sequences.
    args=training_args,
)

print("Starting training...")

trainer.train()

# Save trained model
save_dir = "Mistral_7B_SFTtrained"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"Trained model has been saved to {save_dir}")

Map (num_proc=2):   0%|          | 0/51760 [00:00<?, ? examples/s]

Starting training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 19,410
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.241500
2,1.673200
3,1.372300
4,1.355000
5,1.145200
6,1.048100
7,0.753400
8,0.960500
9,0.899300
10,0.893600


KeyboardInterrupt: 

## Testing after training

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

# Set model device to 'cuda'
torch.set_default_device("cuda")

# Initialize model and corresponding tokenizer
tokenizer = AutoTokenizer.from_pretrained("./Mistral_7B_SFTtrained")
model = AutoModelForCausalLM.from_pretrained("./Mistral_7B_SFTtrained")


In [ ]:
import numpy as np

prompt = "There is most likely going to be fog around\n A. a marsh B. a tundra C. the plains D. a desert\n Respond and give your confidence in percentage terms. In the format, Answer-(a letter option) and Confidence-(a percent figure)"

inputs = tokenizer(prompt, return_tensors="pt")
inputs = inputs.to("cuda")

output = model.generate(**inputs, max_new_tokens=50, 
                         do_sample=False, 
                         temperature=1.0, 
                         output_logits=True,
                         return_dict_in_generate=True)

logits = output.logits

input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]

generated_tokens = output.sequences[:, input_length:]

# Apply softmax function to logits tensors
log_probs = torch.stack(logits, dim=1).softmax(-1)

# Extract probability that corresponds with generated tokens based on the softmax output
gen_probs = torch.gather(log_probs, 2, generated_tokens[:, :, None]).squeeze(-1)

token_probs = {}

for tok, score in zip(generated_tokens[0], gen_probs[0]):
    # Format token string | probability
    # print(f" {tokenizer.decode(tok)} | {score.cpu().numpy()}")
    token_probs[tokenizer.decode(tok)] = score.cpu().numpy() * 100
    
output_answer = tokenizer.batch_decode(output.sequences, skip_special_tokens=True)[0]

print(output_answer)